In [1]:
import glob
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
#UTILITY FUNCTIONS
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df



# BUILDING jobs_data.csv FILE (all data related to each launched job)

In [3]:
path = r'./jobs_data/cnd1/' 
all_files = glob.glob(os.path.join(path , "*.txt"))
li = []
for filename in all_files:
    print("Working on: "+filename)
    df = pd.read_csv(filename, index_col="jobid",header=None,sep=r"\s+", names=["jobid", "jobname","start_ts", "end_ts", "nodes"])
    li.append(df)

df = pd.concat(li, axis=0)#, ignore_index=True)
df['condition_n']=1
print("Found "+str(len(df))+ " jobs with condition number 1")

path = r'./jobs_data/cnd1000/' 
all_files = glob.glob(os.path.join(path , "*.txt"))
li = []
for filename in all_files:
    print("Working on: "+filename)
    df1000 = pd.read_csv(filename, index_col="jobid",header=None,sep=r"\s+", names=["jobid", "jobname","start_ts", "end_ts", "nodes"])
    li.append(df1000)

df1000 = pd.concat(li, axis=0)#, ignore_index=True)
df1000['condition_n']=1000
print("Found "+str(len(df1000))+ " jobs with condition number 1000")

df = pd.concat([df,df1000], axis=0)

df.head()



Working on: ./jobs_data/cnd1/20231203_jobs.txt
Working on: ./jobs_data/cnd1/20231202_jobs.txt
Working on: ./jobs_data/cnd1/20231124_jobs.txt
Working on: ./jobs_data/cnd1/20231125_jobs.txt
Working on: ./jobs_data/cnd1/20240102_jobs.txt
Working on: ./jobs_data/cnd1/20240103_jobs.txt
Working on: ./jobs_data/cnd1/20231227_jobs.txt
Working on: ./jobs_data/cnd1/20231129_jobs.txt
Working on: ./jobs_data/cnd1/20231128_jobs.txt
Working on: ./jobs_data/cnd1/20231204_jobs.txt
Working on: ./jobs_data/cnd1/20231205_jobs.txt
Working on: ./jobs_data/cnd1/20231123_jobs.txt
Working on: ./jobs_data/cnd1/20231122_jobs.txt
Working on: ./jobs_data/cnd1/20231130_jobs.txt
Working on: ./jobs_data/cnd1/20231229_jobs.txt
Working on: ./jobs_data/cnd1/20231228_jobs.txt
Working on: ./jobs_data/cnd1/20231201_jobs.txt
Working on: ./jobs_data/cnd1/20231127_jobs.txt
Working on: ./jobs_data/cnd1/20231126_jobs.txt
Working on: ./jobs_data/cnd1/20231121_jobs.txt
Found 3612 jobs with condition number 1
Working on: ./jobs_d

,jobname,start_ts,end_ts,nodes,condition_n
jobid,,,,,
891576,SPKFT2_nf2_cp576_tnp578_ms42240_double_lbn_r1_...,1701589448,1701589519,48*cresco6x051:48*cresco6x114:48*cresco6x147:4...,1
891577,IMeCOFT4_nf0_cp576_tnp672_ms42240_double_lbn_r...,1701589543,1701590227,48*cresco6x214:48*cresco6x113:48*cresco6x230:4...,1
891578,SPKFT4_nf0_cp576_tnp580_ms42240_double_lbn_r1_...,1701590250,1701590319,48*cresco6x299:48*cresco6x246:48*cresco6x221:4...,1
891579,IMeCOFT4_nf4_cp576_tnp672_ms42240_double_lbn_r...,1701590345,1701590957,48*cresco6x229:48*cresco6x303:48*cresco6x113:4...,1
891580,SPKFT4_nf4_cp576_tnp580_ms42240_double_lbn_r1_...,1701590976,1701591048,48*cresco6x188:48*cresco6x062:48*cresco6x081:4...,1


In [4]:
#creating a colum to report termination with/without error
df['error_code']=0

for index, row in df.iterrows():
    foundErr=False
    for dirpath, dirnames, filenames in os.walk("./results/"):
        for filename in [f for f in filenames if f==(row['jobname']+".err")]:
            errorfile=os.path.join(dirpath, filename)
            csvfile=errorfile.replace("err","csv")
            foundErr=True
            break
        if foundErr==True:
            break

    if not os.path.exists(errorfile): #err file does not exist
        print("jobid="+str(index)+": No .err file for jobname="+row['jobname']+". Setting to 1.")
        df.at[index, 'error_code'] = 1
    elif os.path.getsize(errorfile)!=0: #err file exist and has content
        print("jobid="+str(index)+": Errorfile has content for jobname="+row['jobname'])
        df.at[index, 'error_code'] = 1

    if not os.path.exists(csvfile):
        print("jobid="+str(index)+": No .csv file for jobname="+row['jobname']+".")
        if df.at[index, 'error_code']==0:
            print("error_code was 0. Setting to 1.")
            df.at[index, 'error_code'] = 1
    else: #csv file exist
        with open(csvfile, "r") as f:
            for line in f:
                pass
            try:
                csv_error_code = int(line.split(',')[3])
            except IndexError:
                print("jobid="+str(index)+": No standard format for csvfile="+csvfile) 
                continue
            if csv_error_code != 0 :
                #print("Last line is "+line)
                #print("jobid="+str(index)+": Csvfile reports csv_error_code="+str(csv_error_code)+" for jobname="+row['jobname']+". Overriding error_code="+str(df.at[index, 'error_code']))
                df.at[index, 'error_code'] = csv_error_code
         
    
df[df['error_code'] != 0].head()

jobid=891582: Errorfile has content for jobname=SPKFT8_nf0_cp576_tnp584_ms42240_double_lbn_r1_W15_20231130_023623
jobid=881812: Errorfile has content for jobname=IMeCOFT2_nf2_cp484_tnp528_ms15840_double_lbn_r1_W3_20231123_005128
jobid=881812: No .csv file for jobname=IMeCOFT2_nf2_cp484_tnp528_ms15840_double_lbn_r1_W3_20231123_005128.
jobid=937422: Errorfile has content for jobname=IMeCOFT4_nf4_cp256_tnp320_ms42240_single_lby_r1_W40_20231226_165115
jobid=887340: Errorfile has content for jobname=SPKFT1_nf1_cp484_tnp485_ms26400_double_lbn_r1_W7_20231126_083057
jobid=881573: Errorfile has content for jobname=SPKFT1_nf0_cp400_tnp401_ms15840_single_lbn_r1_W3_20231123_005057
jobid=880090: Errorfile has content for jobname=IMeCOFT1_nf0_cp144_tnp156_ms26400_double_lbn_r1_W20_20231121_222707
jobid=890918: Errorfile has content for jobname=IMeCOFT4_nf4_cp400_tnp480_ms36960_double_lby_r1_W10_20231129_150148
jobid=890920: Errorfile has content for jobname=IMeCOFT8_nf0_cp400_tnp560_ms36960_double_l

,jobname,start_ts,end_ts,nodes,condition_n,error_code
jobid,,,,,,
891576,SPKFT2_nf2_cp576_tnp578_ms42240_double_lbn_r1_...,1701589448,1701589519,48*cresco6x051:48*cresco6x114:48*cresco6x147:4...,1,-99
891580,SPKFT4_nf4_cp576_tnp580_ms42240_double_lbn_r1_...,1701590976,1701591048,48*cresco6x188:48*cresco6x062:48*cresco6x081:4...,1,-99
891582,SPKFT8_nf0_cp576_tnp584_ms42240_double_lbn_r1_...,1701591716,1701591788,48*cresco6x018:48*cresco6x062:48*cresco6x081:4...,1,1
891584,SPKFT8_nf8_cp576_tnp584_ms42240_double_lbn_r1_...,1701592504,1701592573,48*cresco6x206:48*cresco6x225:48*cresco6x074:4...,1,-99
891590,SPKFT1_nf1_cp576_tnp577_ms42240_single_lby_r1_...,1701593721,1701593767,48*cresco6x023:48*cresco6x246:48*cresco6x218:4...,1,-99


In [5]:
#creating a colum to report the run time (whole time by LSF, including process dipatching and termination)
df['runtime']=0
#creating a colum to report the run time of the routine call (whole time, including inizialization of the algorithm)
df['init_and_call_runtime']=0
#creating a colum to report the pure run time of the routine call (no inizialization)
df['call_runtime']=0


for index, row in df.iterrows():
    txtfile_found=False
    for dirpath, dirnames, filenames in os.walk("./results/"):
        for filename in [f for f in filenames if f==(row['jobname']+".txt")]:
            txtfile=os.path.join(dirpath, filename)
            txtfile_found=True
            #print("Examining "+row['jobname']+".txt ...")
            break
        if txtfile_found==True:
            break
   
    if txtfile_found==True: 
        with open(txtfile, "r") as searchfile:
            runtime_found=False
            call_runtime_found=False
            for line in searchfile:
                left,sep,right = line.partition('Run time :')
                if sep: 
                    #print("jobid="+str(index)+" jobname="+row['jobname']+" runtime is "+right.split('sec.')[0].strip())
                    df.at[index, 'runtime']=int(right.split('sec.')[0].strip())
                    runtime_found=True
                left,sep,right = line.partition('Call    run time:')
                if sep:
                    both=right.split('s')[0].strip().split()
                    df.at[index, 'init_and_call_runtime']=int(both[0])
                    df.at[index, 'call_runtime']=int(both[1].replace('(','').replace(')',''))
                    #print("jobid="+str(index)+" init_and_call_runtime is "+row['init_and_call_runtime']+" call_runtime is "+row['call_runtime'])
                    call_runtime_found=True
                    
            if runtime_found==False and row['error_code']==0:
                    print("jobid="+str(index)+": No 'Run time :' string in file: "+txtfile+" row['error']="+str(row['error']))
            if call_runtime_found==False and row['error_code']==0:
                print("jobid="+str(index)+": No 'Call    run time:' string in file: "+txtfile+" row['error']="+str(row['error']))
    
    if txtfile_found==False:
        print("jobid="+str(index)+": No .txt file for jobid="+str(index)+" jobname="+row['jobname'])
        #row['error']=1
df.head()


jobid=921981: No .txt file for jobid=921981 jobname=IMeCOFT8_nf0_cp100_tnp180_ms21120_single_lby_r1_W20_20231214_172656


,jobname,start_ts,end_ts,nodes,condition_n,error_code,runtime,init_and_call_runtime,call_runtime
jobid,,,,,,,,,
891576,SPKFT2_nf2_cp576_tnp578_ms42240_double_lbn_r1_...,1701589448,1701589519,48*cresco6x051:48*cresco6x114:48*cresco6x147:4...,1,-99,71,47,32
891577,IMeCOFT4_nf0_cp576_tnp672_ms42240_double_lbn_r...,1701589543,1701590227,48*cresco6x214:48*cresco6x113:48*cresco6x230:4...,1,0,685,659,652
891578,SPKFT4_nf0_cp576_tnp580_ms42240_double_lbn_r1_...,1701590250,1701590319,48*cresco6x299:48*cresco6x246:48*cresco6x221:4...,1,0,69,47,32
891579,IMeCOFT4_nf4_cp576_tnp672_ms42240_double_lbn_r...,1701590345,1701590957,48*cresco6x229:48*cresco6x303:48*cresco6x113:4...,1,0,612,588,581
891580,SPKFT4_nf4_cp576_tnp580_ms42240_double_lbn_r1_...,1701590976,1701591048,48*cresco6x188:48*cresco6x062:48*cresco6x081:4...,1,-99,72,47,32


In [6]:
df.to_csv('jobs_data.csv', float_format='%f',index=False)
print("jobs_data.csv DONE!")


jobs_data.csv DONE!


### OPTIONAL CHECK: difference between RUNTIME and end_ts - start_ts:

In [7]:
for index, row in df.iterrows():
    diff = row['end_ts'] - row['start_ts']
    if abs(diff - row['runtime'])>1 and row['error_code']==0:
        print("jobid="+str(index)+" jobname="+row['jobname']+" diff="+str(diff)+" row['runtime']="+str(row['runtime']))

jobid=884392 jobname=IMeCOFT2_nf0_cp100_tnp120_ms5280_single_lbn_r1_W5_20231125_072341 diff=8 row['runtime']=16
jobid=884600 jobname=IMeCOFT8_nf8_cp64_tnp128_ms10560_double_lby_r1_W8_20231125_072617 diff=62 row['runtime']=71
jobid=884845 jobname=IMeCOFT2_nf2_cp144_tnp168_ms10560_double_lbn_r1_W8_20231125_072650 diff=36 row['runtime']=44
jobid=884848 jobname=SPKFT4_nf0_cp144_tnp148_ms10560_double_lbn_r1_W8_20231125_072650 diff=12 row['runtime']=23
jobid=884852 jobname=SPKFT8_nf0_cp144_tnp152_ms10560_double_lbn_r1_W8_20231125_072651 diff=12 row['runtime']=20
jobid=884856 jobname=SPKFT0_cp144_tnp144_ms10560_single_lby_r1_W8_20231125_072651 diff=8 row['runtime']=17
jobid=942487 jobname=IMeCOFT1_nf1_cp144_tnp156_ms31680_double_lbn_r1_W60_20240102_150203 diff=874 row['runtime']=885
jobid=942499 jobname=IMeCOFT0_cp144_tnp144_ms36960_double_lby_r1_W60_20240102_150204 diff=1380 row['runtime']=1390
jobid=889545 jobname=IMeCOFT8_nf8_cp484_tnp660_ms31680_double_lby_r1_W8_20231128_101631 diff=288 r

In [8]:
df['algorithm']=None
df['fault_tolerance']=None
df['fault_happened']=None
df['computing_proc']=None
df['total_proc']=None
df['matrix_size']=None
df['precision']=None
df['balanced']=None
df['repetition']=None
df['req_whole_time']=None
df['number_of_nodes']=None

for i in range(1,17):
    df['node'+str(i)]=0 
    

for index, row in df.iterrows():
    el=row['jobname'].split('_')
    df.at[index, 'algorithm']=el[0].split('FT')[0]
    df.at[index, 'fault_tolerance']=int(el[0].split('FT')[1])
    if df.at[index, 'fault_tolerance']!=0:
        df.at[index, 'fault_happened']= int(el[1].split('nf')[1])
        count_el=2
    else:
        df.at[index, 'fault_happened']= 0
        count_el=1
    df.at[index, 'computing_proc']=int(el[count_el].split('cp')[1])
    count_el+=1
    df.at[index, 'total_proc']=int(el[count_el].split('tnp')[1])
    count_el+=1
    df.at[index, 'matrix_size']=int(el[count_el].split('ms')[1])
    count_el+=1
    df.at[index, 'precision']=el[count_el] #single or double
    count_el+=1
    df.at[index, 'balanced']=el[count_el].split('lb')[1] #y or n
    count_el+=1
    df.at[index, 'repetition']=int(el[count_el].split('r')[1]) 
    count_el+=1
    df.at[index, 'req_whole_time']=int(el[count_el].split('W')[1]) 

    df.at[index, 'number_of_nodes']=row['nodes'].count('cresco')

    i=1
    for node in df.at[index, 'nodes'].split(':'):
        df.at[index,'node'+str(i)]=int(node.split('*')[0])
        i+=1


df.head()

,jobname,start_ts,end_ts,nodes,condition_n,error_code,runtime,init_and_call_runtime,call_runtime,algorithm,fault_tolerance,fault_happened,computing_proc,total_proc,matrix_size,precision,balanced,repetition,req_whole_time,number_of_nodes,node1,node2,node3,node4,node5,node6,node7,node8,node9,node10,node11,node12,node13,node14,node15,node16
jobid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
891576,SPKFT2_nf2_cp576_tnp578_ms42240_double_lbn_r1_...,1701589448,1701589519,48*cresco6x051:48*cresco6x114:48*cresco6x147:4...,1,-99,71,47,32,SPK,2,2,576,578,42240,double,n,1,15,13,48,48,48,48,48,48,48,48,48,48,48,48,48,0,0,0
891577,IMeCOFT4_nf0_cp576_tnp672_ms42240_double_lbn_r...,1701589543,1701590227,48*cresco6x214:48*cresco6x113:48*cresco6x230:4...,1,0,685,659,652,IMeCO,4,0,576,672,42240,double,n,1,15,14,48,48,48,48,48,48,48,48,48,48,48,48,48,48,0,0
891578,SPKFT4_nf0_cp576_tnp580_ms42240_double_lbn_r1_...,1701590250,1701590319,48*cresco6x299:48*cresco6x246:48*cresco6x221:4...,1,0,69,47,32,SPK,4,0,576,580,42240,double,n,1,15,13,48,48,48,48,48,48,48,48,48,48,48,48,48,0,0,0
891579,IMeCOFT4_nf4_cp576_tnp672_ms42240_double_lbn_r...,1701590345,1701590957,48*cresco6x229:48*cresco6x303:48*cresco6x113:4...,1,0,612,588,581,IMeCO,4,4,576,672,42240,double,n,1,15,14,48,48,48,48,48,48,48,48,48,48,48,48,48,48,0,0
891580,SPKFT4_nf4_cp576_tnp580_ms42240_double_lbn_r1_...,1701590976,1701591048,48*cresco6x188:48*cresco6x062:48*cresco6x081:4...,1,-99,72,47,32,SPK,4,4,576,580,42240,double,n,1,15,13,48,48,48,48,48,48,48,48,48,48,48,48,48,0,0,0


In [9]:
df.to_csv('jobs_data.csv', float_format='%f',index=True)
print("job_data.csv DONE!")


job_data.csv DONE!
